<a href="https://colab.research.google.com/github/amitmankikar/MDP-with-Value-Iteration-and-Policy-Iteration/blob/master/MDP_with_PI_VI_Q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement Learning Overview

It is the area of Machine learning that deal with how an agent behave in an enviorment. This notebook covers two fundamental algorithms to solve MDPs namely **Value Iteration** and **Policy Iteration**.

## Markov Decision Process - MDP

In MDP, there is an agent. The agent choose an action $a_{t}$ at time $t$ and as a consequance the enviorment changes.
Here The evniorment is world around the agent. After the action the enviorment state changes to $s_{t+1}$.
A reward might be emitted assciated with what just happened and then this process repeats. ![](https://github.com/waqasqammar/MDP-with-Value-Iteration-and-Policy-Iteration/blob/master/nb_images/mdp.png?raw=1)

So, there is a feedback cycle in that the next action you take, the next decision you make is in a situation that's the consiquence of what you did before.

## 1. Import libraries

In [0]:
import numpy as np
import gym
import gym.spaces as spaces
import time

In [0]:
# action mapping for display the final result
action_mapping = {
    3: '\u2191', # UP
    2: '\u2192', # RIGHT
    1: '\u2193', # DOWN
    0: '\u2190' # LEFT
}
print(' '.join([action_mapping[i] for i in range(4)]))


action_mapping2 = {
    3: 'U', # UP
    2: 'R', # RIGHT
    1: 'D', # DOWN
    0: 'L' # LEFT
}


← ↓ → ↑


## 2. Setup GYM Env for playing

we define a faction that will take a GYM enviorment and plays number of games according to given policy.

In [0]:
def play_episodes(enviorment, n_episodes, policy, random = False):
    """
    This fucntion plays the given number of episodes given by following a policy or sample randomly from action_space.
    
    Parameters:
        enviorment: openAI GYM object
        n_episodes: number of episodes to run
        policy: Policy to follow while playing an episode
        random: Flag for taking random actions. if True no policy would be followed and action will be taken randomly
        
    Return:
        wins: Total number of wins playing n_episodes
        total_reward: Total reward of n_episodes
        avg_reward: Average reward of n_episodes
    
    """
    # intialize wins and total reward
    wins = 0
    total_reward = 0
    
    # loop over number of episodes to play
    for episode in range(n_episodes):
        
        # flag to check if the game is finished
        terminated = False
        
        # reset the enviorment every time when playing a new episode
        state = enviorment.reset()
        
        while not terminated:
            
            # check if the random flag is not true then follow the given policy other wise take random action
            if random:
                action = enviorment.action_space.sample()
            else:
                action = policy[state]

            # take the next step
            next_state, reward,  terminated, info = enviorment.step(action)
            
            #enviorment.render()
            
            # accumalate total reward
            total_reward += reward
            
            # change the state
            state = next_state
            
            # if game is over with positive reward then add 1.0 in wins
            if terminated and reward == 1.0:
                wins += 1
                
    # calculate average reward
    average_reward = total_reward / n_episodes
    
    return wins, total_reward, average_reward
            

## 3. Solve for Value Iteration.

![](https://github.com/waqasqammar/MDP-with-Value-Iteration-and-Policy-Iteration/blob/master/nb_images/value_iter.png?raw=1)

In [0]:

def one_step_lookahead(env, state, V , discount_factor = 0.99):
    """
    Helper function to  calculate state-value function
    
    Arguments:
        env: openAI GYM Enviorment object
        state: state to consider
        V: Estimated Value for each state. Vector of length nS
        discount_factor: MDP discount factor
        
    Return:
        action_values: Expected value of each action in a state. Vector of length nA
    """
    
    # initialize vector of action values
    action_values = np.zeros(env.nA)
    
    # loop over the actions we can take in an enviorment 
    for action in range(env.nA):
        # loop over the P_sa distribution.
        for probablity, next_state, reward, info in env.P[state][action]:
             #if we are in state s and take action a. then sum over all the possible states we can land into.
            action_values[action] += probablity * (reward + (discount_factor * V[next_state]))
            
    return action_values

In [0]:
def update_policy(env, policy, V, discount_factor):
    
    """
    Helper function to update a given policy based on given value function.
    
    Arguments:
        env: openAI GYM Enviorment object.
        policy: policy to update.
        V: Estimated Value for each state. Vector of length nS.
        discount_factor: MDP discount factor.
    Return:
        policy: Updated policy based on the given state-Value function 'V'.
    """
    
    for state in range(env.nS):
        # for a given state compute state-action value.
        action_values = one_step_lookahead(env, state, V, discount_factor)
        
        # choose the action which maximizez the state-action value.
        policy[state] =  np.argmax(action_values)
        
    return policy
    

In [0]:
def value_iteration(env, discount_factor = 0.999, max_iteration = 1000):
    """
    Algorithm to solve MPD.
    
    Arguments:
        env: openAI GYM Enviorment object.
        discount_factor: MDP discount factor.
        max_iteration: Maximum No.  of iterations to run.
        
    Return:
        V: Optimal state-Value function. Vector of lenth nS.
        optimal_policy: Optimal policy. Vector of length nS.
    
    """
    # intialize value fucntion
    V = np.zeros(env.nS)
    
    # iterate over max_iterations
    for i in range(max_iteration):
        
        #  keep track of change with previous value function
        prev_v = np.copy(V) 
    
        # loop over all states
        for state in range(env.nS):
            
            # Asynchronously update the state-action value
            #action_values = one_step_lookahead(env, state, V, discount_factor)
            
            # Synchronously update the state-action value
            action_values = one_step_lookahead(env, state, prev_v, discount_factor)
            
            # select best action to perform based on highest state-action value
            best_action_value = np.max(action_values)
            
            # update the current state-value fucntion
            V[state] =  best_action_value
            
        # if policy not changed over 10 iterations it converged.
        if i % 10 == 0:
            # if values of 'V' not changing after one iteration
            if (np.all(np.isclose(V, prev_v))):
                print('Value converged at iteration %d' %(i+1))
                break

    # intialize optimal policy
    optimal_policy = np.zeros(env.nS, dtype = 'int8')
    
    # update the optimal polciy according to optimal value function 'V'
    optimal_policy = update_policy(env, optimal_policy, V, discount_factor)
    
    return V, optimal_policy

## Test the Algorithim

In [0]:
enviorment = gym.make('FrozenLake8x8-v0')
tic = time.time()
opt_V, opt_Policy = value_iteration(enviorment.env, max_iteration = 1000)
toc = time.time()
elapsed_time = (toc - tic) * 1000
print (f"Time to converge: {elapsed_time: 0.3} ms")
print('Optimal Value function: ')
print(opt_V.reshape((8, 8)))
print('Final Policy: ')
tmp = opt_Policy.reshape((8, 8))
#print(tmp)

for r in range(len(tmp)):
  print(' '.join([action_mapping2[int(action)] for action in tmp[r]]))  

#print('\t\t\t'.join([action_mapping[int(action)] for action in opt_Policy]))

Value converged at iteration 601
Time to converge:  5.07e+02 ms
Optimal Value function: 
[[0.89251438 0.89520026 0.89919342 0.90376571 0.90866503 0.91368771
  0.9185209  0.92231843]
 [0.8918656  0.8939008  0.89732647 0.90158507 0.90637534 0.91160223
  0.91748525 0.92509147]
 [0.87639611 0.86213473 0.81870388 0.         0.77762511 0.86519756
  0.90896731 0.93064573]
 [0.86356823 0.8113023  0.6991162  0.420497   0.56364163 0.
  0.8815037  0.93899761]
 [0.85334708 0.71065134 0.46945033 0.         0.49449902 0.56831481
  0.799196   0.95017178]
 [0.84570508 0.         0.         0.1526892  0.35302817 0.41295826
  0.         0.96420128]
 [0.84062173 0.         0.164127   0.10549919 0.         0.31877351
  0.         0.98112763]
 [0.83808344 0.61180435 0.38737881 0.         0.27175197 0.54432026
  0.7715021  0.        ]]
Final Policy: 
U R R R R R R R
U U U U U U U R
L U L L R U R R
L L L D L L R R
L U L L R D U R
L L L D U L L R
L L D L L L L R
L D L L D R D L


In [0]:
n_episode = 1000
wins, total_reward, avg_reward = play_episodes(enviorment, n_episode, opt_Policy, random = False)

In [0]:
print(f'Total wins with value iteration: {wins}')
print(f"Average rewards with value iteration: {avg_reward}")

Total wins with value iteration: 847
Average rewards with value iteration: 0.847


## 4. Solve for Policy Iteration

![](https://github.com/waqasqammar/MDP-with-Value-Iteration-and-Policy-Iteration/blob/master/nb_images/policy_iter.png?raw=1)

In [0]:
def policy_eval(env, policy, V, discount_factor):
    """
    Helper function to evaluate a policy.
    
    Arguments:
        env: openAI GYM Enviorment object.
        policy: policy to evaluate.
        V: Estimated Value for each state. Vector of length nS.
        discount_factor: MDP discount factor.
    Return:
        policy_value: Estimated value of each state following a given policy and state-value 'V'. 
        
    """
    policy_value = np.zeros(env.nS)
    for state, action in enumerate(policy):
        for probablity, next_state, reward, info in env.P[state][action]:
            policy_value[state] += probablity * (reward + (discount_factor * V[next_state]))
            
    return policy_value

In [0]:
def policy_iteration(env, discount_factor = 0.9999, max_iteration = 1000):
    """
    Algorithm to solve MPD.
    
    Arguments:
        env: openAI GYM Enviorment object.
        discount_factor: MDP discount factor.
        max_iteration: Maximum No.  of iterations to run.
        
    Return:
        V: Optimal state-Value function. Vector of lenth nS.
        new_policy: Optimal policy. Vector of length nS.
    
    """
    # intialize the state-Value function
    V = np.zeros(env.nS)
    
    # intialize a random policy
    policy = np.random.randint(0, 4, env.nS)
    policy_prev = np.copy(policy)
    
    for i in range(max_iteration):
        
        # evaluate given policy
        V = policy_eval(env, policy, V, discount_factor)
        
        # improve policy
        policy = update_policy(env, policy, V, discount_factor)
        
        # if policy not changed over 10 iterations it converged.
        if i % 10 == 0:
            if (np.all(np.equal(policy, policy_prev))):
                print('policy converged at iteration %d' %(i+1))
                break
            policy_prev = np.copy(policy)
            

            
    return V, policy

## Test Policy Iteration

In [0]:
enviorment2 = gym.make('FrozenLake8x8-v0')
tic = time.time()
opt_V2, opt_policy2 = policy_iteration(enviorment2.env, discount_factor = 0.999, max_iteration = 10000)
toc = time.time()
elapsed_time = (toc - tic) * 1000
print (f"Time to converge: {elapsed_time: 0.3} ms")
print('Optimal Value function: ')
print(opt_V2.reshape((8, 8)))
print('Final Policy: ')
#print(opt_policy2)
#print(' '.join([action_mapping[(action)] for action in opt_policy2]))

tmp = opt_policy2.reshape((8, 8))
#print(tmp)

for r in range(len(tmp)):
  print(' '.join([action_mapping2[int(action)] for action in tmp[r]]))  

#print('\t\t\t'.join([action_mapping[int(action)] for action in opt_Policy]))

policy converged at iteration 71
Time to converge:  59.0 ms
Optimal Value function: 
[[0.41662507 0.44211228 0.4790079  0.51930038 0.55941994 0.59533683
  0.6186486  0.62520455]
 [0.41036321 0.42979845 0.46192668 0.50079507 0.54343069 0.59046392
  0.6300175  0.64290514]
 [0.38711423 0.38955309 0.37656062 0.         0.47308683 0.56398111
  0.64566115 0.67252113]
 [0.35084055 0.33942226 0.30107344 0.2064086  0.32890866 0.
  0.6503424  0.71476906]
 [0.29777629 0.26746406 0.1851217  0.         0.31994334 0.40543279
  0.60489118 0.77107742]
 [0.23137163 0.         0.         0.09117882 0.23524157 0.3011139
  0.         0.83938889]
 [0.18751728 0.         0.03479682 0.04123882 0.         0.26924241
  0.         0.91705258]
 [0.16575624 0.10868456 0.06701109 0.         0.25414113 0.50993283
  0.75389126 0.        ]]
Final Policy: 
U R R R R R R R
U U U U U R R D
U U L L R U R D
U U U D L L R R
U U L L R D U R
L L L D U L L R
L L D L L L L R
L D L L D R D L


In [0]:
n_episode = 1000
wins, total_reward, avg_reward = play_episodes(enviorment2, n_episode, opt_policy2, random = False)

In [0]:
print(f'Total wins with Policy iteration: {wins}')
print(f"Average rewards with Policy iteration: {avg_reward}")

Total wins with Policy iteration: 853
Average rewards with Policy iteration: 0.853


## Remarks

Policy Iteration converge faster but takes more computation.

In [0]:
# https://github.com/Lodur03/Q-Learning-Taxi-v2/blob/master/Q-Learning-Taxi.ipynb

from IPython.display import clear_output
import numpy as np
import random
import time
import gym

env = gym.make("FrozenLake8x8-v0") # Create environment
env.render() # Show it


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [0]:
# Number of possible actions
action_size = env.action_space.n 
print("Action size ", action_size) 

# Number of possible states
state_size = env.observation_space.n 
print("State size ", state_size)


Action size  4
State size  64


In [0]:
qtable = np.zeros((state_size, action_size))
print(qtable.shape)

(64, 4)


In [0]:
episodes = 30000            # Total episodes
max_steps = 1000            # Max steps per episode
lr = 0.3                    # Learning rate
decay_fac = 0.00001         # Decay learning rate each iteration
gamma = 0.90                # Discounting rate - later rewards impact less

In [0]:
for episode in range(episodes):
    
    state = env.reset() # Reset the environment
    done = False        # Are we done with the environment
    lr -= decay_fac     # Decaying learning rate
    step = 0
    
    if lr <= 0: # Nothing more to learn?
        break
        
    for step in range(max_steps):
        
        # Randomly Choose an Action
        action = env.action_space.sample()
        
        # Take the action -> observe new state and reward
        new_state, reward, done, info = env.step(action)
        
        # Update qtable values
        if done == True: # If last, do not count future accumulated reward
            if(step < 199 | step > 201):
                qtable[state, action] = qtable[state, action]+lr*(reward+gamma*0-qtable[state,action])
            break
        else: # Consider accumulated reward of best decision stream
            qtable[state, action] = qtable[state,action]+lr*(reward+gamma*np.max(qtable[new_state,:])-qtable[state,action])
    
        # if done.. jump to next episode
        if done == True:
            break
        
        # moving states
        state = new_state
        
    episode += 1
    
    if (episode % 3000 == 0):
        print('episode = ', episode)
        print('learning rate = ', lr)
        print('-----------')

episode =  3000
learning rate =  0.26999999999997
-----------
episode =  6000
learning rate =  0.23999999999993998
-----------
episode =  9000
learning rate =  0.20999999999990998
-----------
episode =  12000
learning rate =  0.17999999999987998
-----------
episode =  15000
learning rate =  0.14999999999984998
-----------
episode =  18000
learning rate =  0.11999999999982693
-----------
episode =  21000
learning rate =  0.08999999999983856
-----------
episode =  24000
learning rate =  0.059999999999848445
-----------
episode =  27000
learning rate =  0.029999999999839697
-----------


In [0]:
# New environment
state = env.reset()
env.render()
done = False
total_reward = 0

while(done == False):
    
    action = np.argmax(qtable[state,:]) # Choose best action (Q-table)
    state, reward, done, info = env.step(action) # Take action
    total_reward += reward  # Summing rewards
    
    # Display it
    time.sleep(0.5)
    clear_output(wait=True)
    env.render()
    print('Episode Reward = ', total_reward)


  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Episode Reward =  1.0


In [0]:
#print(qtable.shape)

tmp1 = np.zeros(qtable.shape[0])

for state in range(qtable.shape[0]):
  tmp1[state] = int(np.argmax( qtable[state,:] ) )
                
tmp1 = tmp1.reshape(8, 8)

for r in range(len(tmp1[0])):
  print(' '.join([action_mapping2[int(action)] for action in tmp1[r]]))  


U R R R R R R R
U R U U R R R D
U U L L R U R D
U U U D L L R D
U U L L R D R R
L L L D U L L R
L L D R L R L R
L D L L R R U L
